##### Copyright 2025 Google LLC.

In [ ]:
# @title Licenciado bajo la Licencia Apache, Versión 2.0 (la "Licencia");
# no puedes usar este archivo excepto en cumplimiento con la Licencia.
# Puedes obtener una copia de la Licencia en
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# A menos que lo requiera la ley aplicable o se acuerde por escrito, el software
# distribuido bajo la Licencia se distribuye "TAL CUAL", SIN GARANTÍAS NI CONDICIONES
# de ningún tipo, ya sean expresas o implícitas.
# Consulta la Licencia para conocer el lenguaje específico que rige los permisos y
# limitaciones bajo la Licencia.

# Día 4 - Búsqueda en Google con Grounding usando la API de Gemini

¡Bienvenido de nuevo al curso de Generative AI de 5 días en Kaggle!

En este notebook opcional, utilizarás los resultados de [Google Search](https://google.com/) con la API de Gemini en una técnica llamada grounding, donde el modelo se conecta a fuentes verificables de información. Usar grounding con búsqueda es similar a usar el sistema RAG que implementaste anteriormente en la semana, pero la API de Gemini automatiza gran parte del proceso por ti. El modelo genera consultas de búsqueda en Google y realiza las búsquedas automáticamente, recuperando datos relevantes del índice web de Google y proporcionando enlaces a sugerencias de búsqueda que respaldan la consulta, para que tus usuarios puedan verificar las fuentes.

## Novedades en Gemini 2.0

Gemini 2.0 Flash ofrece una generosa cuota de búsquedas en Google como parte del [nivel gratuito](https://ai.google.dev/pricing). Si cambias de modelo a la versión 1.5, necesitarás [habilitar la facturación](https://aistudio.google.com/apikey) para usar Grounding con Google Search, o puedes [probarlo en AI Studio](https://aistudio.google.com/). Consulta las [versiones anteriores de este notebook](https://www.kaggle.com/code/markishere/day-4-google-search-grounding?scriptVersionId=207458162) para obtener orientación.

## Opcional: Usa Google AI Studio

Si deseas probar el grounding con Google Search, sigue esta sección para probarlo usando la interfaz de AI Studio. O salta a la sección `API` para probar la funcionalidad aquí en tu notebook.

### Abrir AI Studio

Comienza yendo a [AI Studio](https://aistudio.google.com/prompts/new_chat). Deberías estar en la interfaz de "Nuevo chat".

El grounding de búsqueda funciona mejor con `gemini-2.0-flash`, pero también puedes probar `gemini-1.5-flash`.

![Nuevo chat en AI Studio](https://storage.googleapis.com/generativeai-downloads/kaggle/ais-newchat.png)

### Haz una pregunta

Ahora ingresa un prompt en la interfaz de chat. Intenta preguntar algo que sea actual y que pueda requerir información reciente para responder, como el resultado de un deporte reciente. Para esta consulta, el grounding estará **deshabilitado** por defecto.

Esta captura de pantalla muestra la respuesta para `¿Cuáles fueron los disfraces de Halloween más populares este año?`. Cada ejecución será diferente, pero típicamente el modelo habla sobre 2023 y da respuestas generales en lugar de respuestas específicas, ya que no tiene acceso a información específica.

![Ejemplo de pregunta-respuesta sin grounding](https://storage.googleapis.com/generativeai-downloads/kaggle/cricket-ungrounded.png)

### Habilitar grounding

En la barra lateral derecha, en la sección `Herramientas`. Encuentra y habilita la opción `Grounding`.

![Botón para habilitar grounding](https://storage.googleapis.com/generativeai-downloads/kaggle/enable-grounding.png)

Ahora vuelve a ejecutar tu pregunta pasando el cursor sobre el prompt del usuario en el historial del chat y presionando el ícono de Gemini ✨ para volver a ejecutar tu prompt.

![Botón para volver a ejecutar el prompt](https://storage.googleapis.com/generativeai-downloads/kaggle/re-run-button.png)

Ahora deberías ver una respuesta generada que hace referencia a fuentes de Google Search.

![Respuesta con fuentes fundamentadas de Google!](https://storage.googleapis.com/generativeai-downloads/kaggle/cricket-grounded.png)

### Prueba tus propias consultas

Explora esta interfaz y prueba otras consultas. ¡Comparte lo que funciona bien en el [Discord](https://discord.com/channels/1101210829807956100/1303438361117069363)! Puedes comenzar desde [esta plantilla en blanco](https://aistudio.google.com/app/prompts/1FZtxKLFZIJ1p_0rICu8K2CNIF1tkAnf4) que tiene el grounding de búsqueda habilitado.

Los pasos restantes requieren una clave de API con facturación habilitada. No son necesarios para completar este curso; si has probado el grounding en AI Studio, has terminado con este notebook.


## Usar la API

Comienza instalando e importando el SDK de Python de la API de Gemini.

In [ ]:
# Desinstalar paquetes de la imagen base de Kaggle que no son necesarios.
!pip uninstall -qy jupyterlab jupyterlab-lsp
# Instalar el SDK de google-genai para este laboratorio.
!pip install -qU 'google-genai==1.7.0'

In [4]:
from google import genai
from google.genai import types

from IPython.display import Markdown, HTML, display

genai.__version__

### Configura tu clave API

Para ejecutar la siguiente celda, tu clave API debe estar almacenada como un [secreto de Kaggle](https://www.kaggle.com/discussions/product-feedback/114053) llamado `GOOGLE_API_KEY`.

Si aún no tienes una clave API, puedes obtener una desde [AI Studio](https://aistudio.google.com/app/apikey). Puedes encontrar [instrucciones detalladas en la documentación](https://ai.google.dev/gemini-api/docs/api-key).

Para hacer que la clave esté disponible a través de los secretos de Kaggle, selecciona `Secrets` en el menú `Add-ons` y sigue las instrucciones para agregar tu clave o habilitarla para este notebook.

In [5]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

client = genai.Client(api_key=GOOGLE_API_KEY)

Si recibiste un mensaje de error como `No user secrets exist for kernel id ...`, entonces necesitas agregar tu clave API a través de `Add-ons`, `Secrets` **y** habilitarla.

![Captura de pantalla de la casilla para habilitar el secreto GOOGLE_API_KEY](https://storage.googleapis.com/kaggle-media/Images/5gdai_sc_3.png)

### Automated retry

In [6]:
# Define a retry policy. The model might make multiple consecutive calls automatically
# for a complex query, this ensures the client retries if it hits quota limits.
from google.api_core import retry

is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

if not hasattr(genai.models.Models.generate_content, '__wrapped__'):
  genai.models.Models.generate_content = retry.Retry(
      predicate=is_retriable)(genai.models.Models.generate_content)

## Use search grounding

### Model support

Search grounding is available in a limited set of models. Find a model that supports it on [the models page](https://ai.google.dev/gemini-api/docs/models/gemini).

In this guide, you'll use `gemini-2.0-flash`.

### Make a request

To enable search grounding, you specify it as a tool: `google_search`. Like other tools, this is supplied as a parameter in `GenerateContentConfig`, and can be passed to `generate_content` calls as well as `chats.create` (for all chat turns) or `chat.send_message` (for specific turns).


<table align=left>
  <td>
    <a target="_blank" href="https://aistudio.google.com/app/prompts/1GTkO-gH4vd6G7LpBJ6Ay7U1OaJer7yDD"><img src="https://ai.google.dev/site-assets/images/marketing/home/icon-ais.png" style="height: 24px" height=24/> Open in AI Studio</a>
  </td>
</table>

In [16]:
# Ask for information without search grounding.
response = client.models.generate_content(
    model='gemini-2.0-flash',
    contents="When and where is Billie Eilish's next concert?")

Markdown(response.text)

Now try with grounding enabled.

<table align=left>
  <td>
    <a target="_blank" href="https://aistudio.google.com/prompts/14lDR0VjSni6BEUCZUBqj5PzTn3J194Th"><img src="https://ai.google.dev/site-assets/images/marketing/home/icon-ais.png" style="height: 24px" height=24/> Open in AI Studio</a>
  </td>
</table>

In [8]:
# And now re-run the same query with search grounding enabled.
config_with_search = types.GenerateContentConfig(
    tools=[types.Tool(google_search=types.GoogleSearch())],
)

response = client.models.generate_content(
    model='gemini-2.0-flash',
    contents="When and where is Billie Eilish's next concert?",
    config=config_with_search,
)

rc = response.candidates[0]

Markdown(rc.content.parts[0].text)

### Response metadata

When search grounding is used, the model returns extra metadata that includes links to search suggestions, supporting documents and information on how the supporting documents were used.

Each "grounding chunk" represents information retrieved from Google Search that was used in the grounded generation request. Following the URI will take you to the source.

In [9]:
chunks = rc.grounding_metadata.grounding_chunks
for chunk in chunks:
    print(f'{chunk.web.title}: {chunk.web.uri}')

As part of the response, there is a standalone styled HTML content block that you use to link back to relevant search suggestions related to the generation.

In [ ]:
HTML(rc.grounding_metadata.search_entry_point.rendered_content)

The `grounding_supports` in the metadata provide a way for you to correlate the grounding chunks used to the generated output text.

In [11]:
from pprint import pprint

supports = rc.grounding_metadata.grounding_supports
for support in supports:
    pprint(support.to_json_dict())

These supports can be used to highlight text in the response, or build tables of footnotes.

In [12]:
import io

markdown_buffer = io.StringIO()

# Print the text with footnote markers.
markdown_buffer.write("Supported text:\n\n")
for support in supports:
    markdown_buffer.write(" * ")
    markdown_buffer.write(
        response.text[support.segment.start_index : support.segment.end_index]
    )

    for i in support.grounding_chunk_indices:
        chunk = chunks[i].web
        markdown_buffer.write(f"<sup>[{i+1}]</sup>")

    markdown_buffer.write("\n\n")


# And print the footnotes.
markdown_buffer.write("Citations:\n\n")
for i, chunk in enumerate(chunks, start=1):
    markdown_buffer.write(f"{i}. [{chunk.web.title}]({chunk.web.uri})\n")


Markdown(markdown_buffer.getvalue())

## Search with tools

In this example, you'll use enable the Google Search grounding tool and the code generation tool across two steps. In the first step, the model will use Google Search to find the requested information and then in the follow-up question, it generates code to plot the results.

This usage includes textual, visual and code parts, so first define a function to help visualise these.

In [13]:
from IPython.display import display, Image, Markdown

def show_response(response):
    for p in response.candidates[0].content.parts:
        if p.text:
            display(Markdown(p.text))
        elif p.inline_data:
            display(Image(p.inline_data.data))
        else:
            print(p.to_json_dict())
    
        display(Markdown('----'))

Now start a chat asking for some information. Here you provide the Google Search tool so that the model can look up data from Google's Search index.

In [14]:
config_with_search = types.GenerateContentConfig(
    tools=[types.Tool(google_search=types.GoogleSearch())],
    temperature=0.0,
)

chat = client.chats.create(model='gemini-2.0-flash')

response = chat.send_message(
    message="What were the medal tallies, by top-10 countries, for the 2024 olympics?",
    config=config_with_search,
)

show_response(response)

Continuing the chat, now ask the model to convert the data into a chart. The `code_execution` tool is able to generate code to draw charts, execute that code and return the image. You can see the executed code in the `executable_code` part of the response.

Combining results from Google Search with tools like live plotting can enable very powerful use cases that require very little code to run.

In [15]:
config_with_code = types.GenerateContentConfig(
    tools=[types.Tool(code_execution=types.ToolCodeExecution())],
    temperature=0.0,
)

response = chat.send_message(
    message="Now plot this as a seaborn chart. Break out the medals too.",
    config=config_with_code,
)

show_response(response)

## Further reading

When using search grounding, there are some specific requirements that you must follow, including when and how to show search suggestions, and how to use the grounding links.  Be sure to read and follow the details in the [search grounding capability guide](https://ai.google.dev/gemini-api/docs/grounding) and the [search suggestions guide](https://ai.google.dev/gemini-api/docs/grounding/search-suggestions).

Also check out some more compelling examples of using search grounding with the Live API in the [cookbook](https://github.com/google-gemini/cookbook/), like [this example that uses Google Maps to plot Search results on a map](https://github.com/google-gemini/cookbook/blob/main/examples/LiveAPI_plotting_and_mapping.ipynb) in an audio conversation, or [this example](https://github.com/google-gemini/cookbook/blob/main/examples/Search_grounding_for_research_report.ipynb) that builds a comprehensive research report.

*- [Mark McD](https://linktr.ee/markmcd)*